In [ ]:
%pip install pandas requests lxml tqdm

Note: you may need to restart the kernel to use updated packages.


중앙부처

# 확장된 코드 성공적

In [22]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import ssl
import time
from requests.adapters import HTTPAdapter
import os

# --- 사용자 입력 필요 ---
API_KEY = "1e69f4a6e721b5a73162d8228932fec131a13601cb7d2376e94c45d98516dcbd"
CSV_FILE_NAME = "중앙부처_복지서비스.csv"   # 서비스 ID 목록 CSV
SERVICE_ID_COLUMN = "서비스ID"
BASE_URL = "https://apis.data.go.kr/B554287/NationalWelfareInformationsV001/NationalWelfaredetailedV001"

# TLS 강제
class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.minimum_version = ssl.TLSVersion.TLSv1_2
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

session = requests.Session()
session.mount("https://", TLSAdapter())


# -------------------------------
# XML 파싱
# -------------------------------
def parse_service_detail(xml_text):
    try:
        root = ET.fromstring(xml_text)

        # 루트가 <wantedDtl>일 수 있음
        if root.tag == "wantedDtl":
            node = root
        else:
            node = root.find(".//wantedDtl")

        if node is None:
            print("⚠️ <wantedDtl> 태그를 찾을 수 없습니다.")
            return None

        row = {
            "서비스ID": node.findtext("servId", ""),
            "서비스명": node.findtext("servNm", ""),
            "소관부처": node.findtext("jurMnofNm", ""),
            "서비스개요": node.findtext("wlfareInfoOutlCn", ""),
            "지원대상상세": node.findtext("tgtrDtlCn", "").strip(),
            "선정기준": node.findtext("slctCritCn", "").strip(),
            "지원내용": node.findtext("alwServCn", "").strip(),
            "지원주기": node.findtext("sprtCycNm", ""),
            "지급방식": node.findtext("srvPvsnNm", ""),
        }
        return row

    except Exception as e:
        print(f"⚠️ 파싱 오류: {e}")
        return None
# -------------------------------
# api 호출 함수
# -------------------------------
def fetch_service_detail(sid):
    params = {"serviceKey":API_KEY, "servId":sid}
    try:
        resp = session.get(BASE_URL, params=params, timeout=30)
        print(f"응답코드: {resp.status_code}")
        
        if resp.status_code == 200 :
            return resp.text
        else :
            print(f"요청실패:{resp.status_code}")
            
            return None
    except Exception as e:
        print(f"에러요청:{e}")
        
        return None    
# -------------------------------
# 3. 실행 루프
# -------------------------------
def run_batch(start_idx=0, batch_size=100, batch_num=1):
    df = pd.read_csv(CSV_FILE_NAME)
    service_ids = df[SERVICE_ID_COLUMN].dropna().astype(str).tolist()

    # 범위 자르기
    target_ids = service_ids[start_idx:start_idx + batch_size]
    print(f"\n✅ 총 {len(target_ids)}개 요청 시작 (Batch {batch_num})")

    rows, success_ids, fail_ids = [], [], []
    last_id = None 
    
    for idx, sid in enumerate(target_ids, start=1):
        print(f"[{idx}/{len(target_ids)}] 요청: {sid}")
        
        try:
            xml_text = fetch_service_detail(sid)
            
            if xml_text == "LIMIT_EXCEEDED":
                print("API 한도 도달 -> 중단 및 저장")
                break
            
            if not xml_text:
                fail_ids.append(sid)
                continue
            
            data = parse_service_detail(xml_text)
            if data:
                rows.append(data)
                success_ids.append(sid)
                last_id = sid
                print(f"   ✅ {sid} 저장됨")
            
            else:
                fail_ids.append(sid)
                print(f"   ✅ {sid} 파싱 실패")
                
        except Exception as e:
            print(f"    ❌ 요청 실패: {e}")
            fail_ids.append(sid)
        
        time.sleep(0.5)

    # 결과 저장
    if rows:
        file_name = f"중앙부처_{batch_num}.csv"
        pd.DataFrame(rows).to_csv(file_name, index=False, encoding="utf-8-sig")
        print(f"\n💾 저장 완료 → {file_name}")
        print(f"👉 마지막 처리된 서비스ID: {last_id}")
    else:
        print("⚠️ 저장할 데이터가 없습니다.")

    if fail_ids:
        fail_file = f"중앙부처_{batch_num}_실패.csv"
        pd.DataFrame(fail_ids, columns=["실패ID"]).to_csv(fail_file, index=False, encoding="utf-8-sig")
        print(f"⚠️ 실패한 ID {len(fail_ids)}건 저장됨 → {fail_file}")


def get_next_start_idx(master_csv, processed_csv_pattern="중앙부처_"):
    df_master = pd.read_csv(master_csv)
    service_ids = df_master["서비스ID"].dropna().astype(str).tolist()

    # 이미 저장된 파일들 찾기
    processed_files = [f for f in os.listdir() if f.startswith(processed_csv_pattern) and f.endswith(".csv")]
    if not processed_files:
        print("✅ 저장된 파일 없음 → 처음부터 시작 (start_idx=0)")
        return 0, 1   # start_idx=0, batch_num=1

    # 가장 마지막 batch 파일 찾기
    processed_files.sort()
    last_file = processed_files[-1]
    print(f"📂 마지막 저장된 파일: {last_file}")

    df_last = pd.read_csv(last_file)
    if df_last.empty:
        print("⚠️ 마지막 파일이 비어 있음 → 처음부터 재시작 필요")
        return 0, 1

    last_id = df_last["서비스ID"].iloc[-1]  # 마지막 저장된 서비스ID
    print(f"👉 마지막 처리된 서비스ID: {last_id}")

    try:
        last_idx = service_ids.index(str(last_id))
    except ValueError:
        print("⚠️ 원본 CSV에서 마지막 ID를 찾지 못함 → 수동으로 지정 필요")
        return 0, 1

    next_idx = last_idx + 1
    next_batch_num = len(processed_files) + 1

    print(f"➡️ 다음 실행 시작 index: {next_idx}, Batch 번호: {next_batch_num}")
    return next_idx, next_batch_num

# -------------------------------
# 실행 예시
# -------------------------------
if __name__ == "__main__":
    master_csv = "중앙부처_복지서비스.csv"

    start_idx, batch_num = get_next_start_idx(master_csv)
    run_batch(start_idx=start_idx, batch_size=100, batch_num=batch_num)


📂 마지막 저장된 파일: 중앙부처_복지서비스_상세_전체결과.csv
👉 마지막 처리된 서비스ID: WLF00003174
➡️ 다음 실행 시작 index: 101, Batch 번호: 10

✅ 총 100개 요청 시작 (Batch 10)
[1/100] 요청: WLF00001187
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00001187 파싱 실패
[2/100] 요청: WLF00003220
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00003220 파싱 실패
[3/100] 요청: WLF00000067
응답코드: 200
   ✅ WLF00000067 저장됨
[4/100] 요청: WLF00000065
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00000065 파싱 실패
[5/100] 요청: WLF00003195
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00003195 파싱 실패
[6/100] 요청: WLF00003279
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00003279 파싱 실패
[7/100] 요청: WLF00001109
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00001109 파싱 실패
[8/100] 요청: WLF00005445
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00005445 파싱 실패
[9/100] 요청: WLF00000032
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00000032 파싱 실패
[10/100] 요청: WLF00003198
응답코드: 200
⚠️ <wantedDtl> 태그를 찾을 수 없습니다.
   ✅ WLF00003198 파싱 실패
[11/100] 요청: WLF00001021
응답코드: 

KeyboardInterrupt: 

# 중앙부처 크롤링 - 공공데이터 포털

In [ ]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import ssl
import time
import os
from requests.adapters import HTTPAdapter

# ==========================
# 사용자 입력
# ==========================
API_KEY = "6c00ed313ec5456c58b8d55a6f5d12a65cd4984956e87be30c7a3ea22b8ca086"   # ← 반드시 본인 API Key로 교체
MASTER_CSV = "중앙부처_복지서비스.csv"     # 전체 서비스ID 목록
PROCESSED_CSV = "중앙부처_batch4.csv"    # 이미 처리된 결과 CSV
SERVICE_ID_COLUMN = "서비스ID"
BASE_URL = "https://apis.data.go.kr/B554287/NationalWelfareInformationsV001/NationalWelfaredetailedV001"

# ==========================
# TLS 강제
# ==========================
class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.minimum_version = ssl.TLSVersion.TLSv1_2
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

session = requests.Session()
session.mount("https://", TLSAdapter())

# ==========================
# XML 파싱
# ==========================
def parse_service_detail(xml_text):
    try:
        root = ET.fromstring(xml_text)
        node = root if root.tag == "wantedDtl" else root.find(".//wantedDtl")
        if node is None:
            return None
        row = {
            "서비스ID": node.findtext("servId", ""),
            "서비스명": node.findtext("servNm", ""),
            "소관부처": node.findtext("jurMnofNm", ""),
            "서비스개요": node.findtext("wlfareInfoOutlCn", ""),
            "지원대상상세": node.findtext("tgtrDtlCn", "").strip(),
            "선정기준": node.findtext("slctCritCn", "").strip(),
            "지원내용": node.findtext("alwServCn", "").strip(),
            "지원주기": node.findtext("sprtCycNm", ""),
            "지급방식": node.findtext("srvPvsnNm", ""),
        }
        return row
    except:
        return None

# ==========================
# API 호출
# ==========================
def fetch_service_detail(sid):
    params = {"serviceKey": API_KEY, "servId": sid}
    try:
        resp = session.get(BASE_URL, params=params, timeout=30)
        if resp.status_code == 200:
            return resp.text
        else:
            print(f"❌ 요청 실패: {sid}, 코드 {resp.status_code}")
            return None
    except Exception as e:
        print(f"⚠️ 요청 오류: {sid}, {e}")
        return None

# ==========================
# 실행 루프
# ==========================
def run_batch(service_ids, batch_num=1, batch_size=100):
    rows, fail_ids = [], []
    last_id = None

    for idx, sid in enumerate(service_ids, start=1):
        print(f"[{idx}/{len(service_ids)}] 요청: {sid}")
        xml_text = fetch_service_detail(sid)

        if not xml_text:
            fail_ids.append(sid)
            continue

        data = parse_service_detail(xml_text)
        if data:
            rows.append(data)
            last_id = sid
        else:
            fail_ids.append(sid)

        time.sleep(0.5)  # API 부하 방지

        # 100개 단위 저장
        if idx % batch_size == 0:
            file_name = f"중앙부처_batch{batch_num}.csv"
            pd.DataFrame(rows).to_csv(file_name, index=False, encoding="utf-8-sig")
            print(f"💾 Batch {batch_num} 저장 완료 ({len(rows)}개)")
            rows = []
            batch_num += 1

    # 남은 데이터 저장
    if rows:
        file_name = f"중앙부처_batch{batch_num}.csv"
        pd.DataFrame(rows).to_csv(file_name, index=False, encoding="utf-8-sig")
        print(f"💾 Batch {batch_num} 저장 완료 ({len(rows)}개)")

    # 실패 ID 저장
    if fail_ids:
        fail_file = f"중앙부처_batch{batch_num}_실패.csv"
        pd.DataFrame(fail_ids, columns=["실패ID"]).to_csv(fail_file, index=False, encoding="utf-8-sig")
        print(f"⚠️ 실패한 ID {len(fail_ids)}건 저장됨 → {fail_file}")

    print(f"✅ 마지막 처리된 서비스ID: {last_id}")


# ==========================
# 시작 index 계산
# ==========================
def get_remaining_ids(master_csv, processed_csv):
    df_master = pd.read_csv(master_csv)
    df_processed = pd.read_csv(processed_csv)

    all_ids = df_master[SERVICE_ID_COLUMN].dropna().astype(str).tolist()
    done_ids = df_processed[SERVICE_ID_COLUMN].dropna().astype(str).tolist()

    remaining = [sid for sid in all_ids if sid not in done_ids]
    print(f"📊 전체 {len(all_ids)}개 중 {len(done_ids)}개 완료, {len(remaining)}개 남음")
    return remaining


# ==========================
# 실행
# ==========================
if __name__ == "__main__":
    remaining_ids = get_remaining_ids(MASTER_CSV, PROCESSED_CSV)

    if remaining_ids:
        run_batch(remaining_ids, batch_num=3, batch_size=100)  
        # 👉 batch_num=3부터 시작 (1,2는 이미 완료했으니까)
    else:
        print("🎉 모든 서비스ID 크롤링 완료됨")


📊 전체 368개 중 190개 완료, 178개 남음
[1/178] 요청: WLF00001068
[2/178] 요청: WLF00000026
[3/178] 요청: WLF00000102
[4/178] 요청: WLF00003265
[5/178] 요청: WLF00004661
[6/178] 요청: WLF00001164
[7/178] 요청: WLF00003252
[8/178] 요청: WLF00001115
[9/178] 요청: WLF00001179
[10/178] 요청: WLF00001133
[11/178] 요청: WLF00003174
[12/178] 요청: WLF00005447
[13/178] 요청: WLF00003188
[14/178] 요청: WLF00001142
[15/178] 요청: WLF00001152
[16/178] 요청: WLF00001139
[17/178] 요청: WLF00000069
[18/178] 요청: WLF00001173
[19/178] 요청: WLF00003251
[20/178] 요청: WLF00001108
[21/178] 요청: WLF00003243
[22/178] 요청: WLF00000098
[23/178] 요청: WLF00001170
[24/178] 요청: WLF00000093
[25/178] 요청: WLF00001079
[26/178] 요청: WLF00005004
[27/178] 요청: WLF00000923
[28/178] 요청: WLF00001169
[29/178] 요청: WLF00000030
[30/178] 요청: WLF00000101
[31/178] 요청: WLF00003182
[32/178] 요청: WLF00003266
[33/178] 요청: WLF00004647
[34/178] 요청: WLF00003241
[35/178] 요청: WLF00000807
[36/178] 요청: WLF00001167
[37/178] 요청: WLF00000068
[38/178] 요청: WLF00001077
[39/178] 요청: WLF00003191
[40/1

# 중앙부처 실패id 합본

In [69]:
import pandas as pd
import glob
import os

# --- 설정 ---

# 1. 찾을 실패 파일들의 패턴입니다. 
#    이 패턴에 맞는 모든 파일을 찾아냅니다. (예: 중앙부처_batch4_실패.csv, 중앙부처_재시도_재실패.csv 등)
FILE_PATTERN = "*_실패.csv"

# 2. 최종적으로 저장될 통합 파일의 이름입니다.
OUTPUT_FILENAME = "통합_실패_ID_목록.csv"

# 3. 통합 후 사용할 최종 서비스 ID 컬럼 이름입니다.
FINAL_ID_COLUMN_NAME = "서비스ID"

# --- 코드 실행 ---

def merge_failure_files():
    """
    지정된 패턴과 일치하는 모든 실패 CSV 파일을 찾아,
    하나의 통합된 파일로 병합하고 중복을 제거합니다.
    """
    # 패턴에 맞는 파일 목록을 찾습니다.
    failure_files = glob.glob(FILE_PATTERN)

    if not failure_files:
        print(f"❌ 오류: '{FILE_PATTERN}' 패턴에 맞는 실패 파일을 찾을 수 없습니다.")
        print("스크립트와 실패 파일들이 같은 폴더에 있는지 확인해주세요.")
        return

    print(f"📁 총 {len(failure_files)}개의 실패 파일을 찾았습니다:")
    for f in failure_files:
        print(f"  - {f}")

    df_list = []
    # 각 파일을 읽어 데이터프레임 리스트에 추가합니다.
    for file in failure_files:
        try:
            df = pd.read_csv(file)
            # 컬럼 이름이 통일되도록 처리
            # 예: '실패ID', '재실패ID' -> '서비스ID'
            if '실패ID' in df.columns:
                df.rename(columns={'실패ID': FINAL_ID_COLUMN_NAME}, inplace=True)
            elif '재실패ID' in df.columns:
                df.rename(columns={'재실패ID': FINAL_ID_COLUMN_NAME}, inplace=True)
            
            df_list.append(df)
        except Exception as e:
            print(f"⚠️ 경고: '{file}' 파일을 읽는 중 오류 발생. 건너뜁니다. ({e})")

    if not df_list:
        print("\n❌ 오류: 파일을 읽어오지 못했습니다.")
        return

    # 모든 데이터프레임을 하나로 합칩니다.
    print("\n파일 병합 및 중복 제거를 시작합니다...")
    combined_df = pd.concat(df_list, ignore_index=True)

    # 최종 ID 컬럼이 있는지 확인하고 중복을 제거합니다.
    if FINAL_ID_COLUMN_NAME in combined_df.columns:
        initial_rows = len(combined_df)
        # 중복된 ID를 제거하고, 최종적으로 유니크한 ID 목록만 남깁니다.
        combined_df.drop_duplicates(subset=[FINAL_ID_COLUMN_NAME], keep='first', inplace=True)
        final_rows = len(combined_df)
        print(f"  - 총 {initial_rows}개의 ID를 {final_rows}개의 고유 ID로 정리했습니다.")
    else:
        print(f"⚠️ 경고: '{FINAL_ID_COLUMN_NAME}' 컬럼을 찾을 수 없어 중복 제거를 건너뜁니다.")
        
    # 최종 결과를 CSV 파일로 저장합니다.
    try:
        combined_df.to_csv(OUTPUT_FILENAME, index=False, encoding='utf-8-sig')
        print(f"\n✅ 성공! '{OUTPUT_FILENAME}' 파일로 모든 실패 ID를 통합 저장했습니다.")
    except Exception as e:
        print(f"\n❌ 오류: 최종 파일을 저장하는 중 문제가 발생했습니다: {e}")


# 메인 함수 실행
if __name__ == "__main__":
    merge_failure_files()

📁 총 2개의 실패 파일을 찾았습니다:
  - 중앙부처_batch4_실패.csv
  - 중앙부처_실패.csv

파일 병합 및 중복 제거를 시작합니다...
  - 총 88개의 ID를 88개의 고유 ID로 정리했습니다.

✅ 성공! '통합_실패_ID_목록.csv' 파일로 모든 실패 ID를 통합 저장했습니다.


# 중앙부처 실패id 재크롤링

In [70]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd
import ssl
import time
from requests.adapters import HTTPAdapter
import os
from tqdm import tqdm

# --- 💡 1. 여기에 모든 API 키를 입력하세요 ---
API_KEYS = [
    "0f561bd1f21a64f960fff4cf634d3afeb996d252a52dae3c7fea3c70c2c9ba09",
    "6c00ed313ec5456c58b8d55a6f5d12a65cd4984956e87be30c7a3ea22b8ca086"
    # 가지고 있는 모든 키를 추가할 수 있습니다.
]

# --- 💡 2. 입력 파일과 결과 파일 이름 설정 ---
INPUT_FILE = "통합_실패_ID_목록.csv"
ID_COLUMN = "서비스ID" # 통합 실패 파일의 ID 컬럼명

OUTPUT_SUCCESS_FILE = "최종_재시도_성공.csv"
OUTPUT_FAILURE_FILE = "최종_재시도_실패.csv"

# --- API 호출 설정 (수정 필요 없음) ---
BASE_URL = "https://www.bokjiro.go.kr/ssis-tbu/TWAT52005M/twataa/wlfareInfo/selectWlfareInfo.do"

# (이 아래 TLSAdapter, session 설정은 기존과 동일합니다)
class TLSAdapter(HTTPAdapter):
    def init_poolmanager(self, *args, **kwargs):
        ctx = ssl.create_default_context(ssl.Purpose.SERVER_AUTH)
        ctx.minimum_version = ssl.TLSVersion.TLSv1_2
        kwargs['ssl_context'] = ctx
        return super().init_poolmanager(*args, **kwargs)

session = requests.Session()
session.mount("https://", TLSAdapter())

# (XML 파싱 함수는 기존과 동일합니다)
def parse_service_detail(xml_text):
    try:
        root = ET.fromstring(xml_text)
        node = root if root.tag == "wantedDtl" else root.find(".//wantedDtl")
        if node is None:
            return None
        return {
            "서비스ID": node.findtext("servId", ""), "서비스명": node.findtext("servNm", ""),
            "소관부처": node.findtext("jurMnofNm", ""), "서비스개요": node.findtext("wlfareInfoOutlCn", ""),
            "지원대상상세": node.findtext("tgtrDtlCn", "").strip(), "선정기준": node.findtext("slctCritCn", "").strip(),
            "지원내용": node.findtext("alwServCn", "").strip(), "지원주기": node.findtext("sprtCycNm", ""),
            "지급방식": node.findtext("srvPvsnNm", ""),
        }
    except Exception:
        return None

def fetch_service_detail(sid, api_key):
    """지정된 API 키로 서비스 상세 정보를 요청합니다."""
    params = {"serviceKey": api_key, "servId": sid}
    try:
        resp = session.get(BASE_URL, params=params, timeout=30)
        # API 한도 초과 에러는 보통 특정 문자열을 포함한 응답을 줍니다.
        # 실제 응답에 따라 'LIMIT_EXCEEDED' 또는 다른 문자열로 변경해야 할 수 있습니다.
        if "SERVICE KEY IS NOT REGISTERED" in resp.text or "LIMITED" in resp.text:
             return "LIMIT_EXCEEDED"
        return resp.text if resp.status_code == 200 else None
    except requests.exceptions.RequestException:
        return None

# --- 최종 실행 로직 ---
def run_final_retry():
    # 1. 재시도할 ID 목록 불러오기
    try:
        df = pd.read_csv(INPUT_FILE)
        service_ids = df[ID_COLUMN].dropna().astype(str).tolist()
    except FileNotFoundError:
        print(f"❌ 입력 파일 '{INPUT_FILE}'을 찾을 수 없습니다. 스크립트와 같은 폴더에 있는지 확인하세요.")
        return

    # 2. 실행 시작
    print(f"✅ 총 {len(service_ids)}개의 실패 ID에 대한 최종 재시도를 시작합니다.")
    print(f"🔑 사용 가능한 API 키: {len(API_KEYS)}개")

    successful_rows = []
    failed_ids = []
    
    current_key_index = 0
    pbar = tqdm(total=len(service_ids), desc="ID 처리 중")
    
    i = 0
    while i < len(service_ids):
        sid = service_ids[i]
        
        if current_key_index >= len(API_KEYS):
            print("\n⚠️ 모든 API 키의 사용 한도에 도달했거나 유효하지 않습니다. 작업을 중단합니다.")
            failed_ids.extend(service_ids[i:]) # 남은 ID는 모두 실패로 처리
            pbar.update(len(service_ids) - i)
            break

        api_key = API_KEYS[current_key_index]
        xml_text = fetch_service_detail(sid, api_key)

        if xml_text == "LIMIT_EXCEEDED":
            print(f"\n🔑 API 키 {current_key_index + 1}의 한도에 도달. 다음 키로 전환합니다.")
            current_key_index += 1
            continue  # 현재 ID(sid)를 다음 키로 다시 시도하기 위해 i를 증가시키지 않음

        elif xml_text is None:
            failed_ids.append(sid)
            pbar.set_postfix_str(f"{sid} 요청 실패")
            i += 1
            pbar.update(1)

        else:
            data = parse_service_detail(xml_text)
            if data:
                successful_rows.append(data)
            else:
                failed_ids.append(sid)
            pbar.set_postfix_str(f"{sid} 처리 완료")
            i += 1
            pbar.update(1)
            
        time.sleep(0.3) # 서버 부하 감소

    pbar.close()

    # 3. 결과 저장
    if successful_rows:
        pd.DataFrame(successful_rows).to_csv(OUTPUT_SUCCESS_FILE, index=False, encoding="utf-8-sig")
        print(f"\n💾 재시도 성공: {len(successful_rows)}건을 '{OUTPUT_SUCCESS_FILE}' 파일로 저장했습니다.")
    
    if failed_ids:
        pd.DataFrame(failed_ids, columns=["재실패ID"]).to_csv(OUTPUT_FAILURE_FILE, index=False)
        print(f"💾 재시도 실패: {len(failed_ids)}건을 '{OUTPUT_FAILURE_FILE}' 파일로 저장했습니다.")
        
    print("\n🎉 모든 작업이 완료되었습니다.")


if __name__ == "__main__":
    run_final_retry()

✅ 총 88개의 실패 ID에 대한 최종 재시도를 시작합니다.
🔑 사용 가능한 API 키: 2개


ID 처리 중: 100%|██████████| 88/88 [00:28<00:00,  3.08it/s, WLF00001164 처리 완료]

💾 재시도 실패: 88건을 '최종_재시도_실패.csv' 파일로 저장했습니다.

🎉 모든 작업이 완료되었습니다.


# 중앙부처csv합치기

In [1]:
import pandas as pd

files = ['중앙부처_1.csv', '중앙부처_2.csv']

dfs = [pd.read_csv(f) for f in files]
merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv("중앙부처_1,2합본.csv", index=False, encoding='utf-8-sig')

print("✅ 통합 완료 → 중앙부처_1,2합본.csv")
print(f"총 행 개수: {len(merged_df)}")
print(f"컬럼: {list(merged_df.columns)}")

FileNotFoundError: [Errno 2] No such file or directory: '중앙부처_1.csv'

중앙부처 합본 -2차

In [71]:
import pandas as pd
import glob
import xml.etree.ElementTree as ET
import os
from tqdm import tqdm

# --- 설정 ---

# 1. 기존에 작업했던 합본 파일 이름
MAIN_CSV_FILE = "중앙부처_1,2합본.csv"

# 2. 폴더에서 찾을 XML 파일 패턴
XML_PATTERN = "*.xml"

# 3. 최종적으로 모든 데이터가 합쳐져 저장될 파일 이름
FINAL_OUTPUT_FILE = "최종_중앙부처_통합본.csv"

# --- XML 파싱 함수 (기존 코드와 동일) ---
def parse_service_detail(xml_text):
    """XML 텍스트를 입력받아 파싱하고, 데이터 딕셔너리를 반환합니다."""
    try:
        root = ET.fromstring(xml_text)
        node = root if root.tag == "wantedDtl" else root.find(".//wantedDtl")
        if node is None:
            return None
        return {
            "서비스ID": node.findtext("servId", ""),
            "서비스명": node.findtext("servNm", ""),
            "소관부처": node.findtext("jurMnofNm", ""),
            "서비스개요": node.findtext("wlfareInfoOutlCn", ""),
            "지원대상상세": node.findtext("tgtrDtlCn", "").strip(),
            "선정기준": node.findtext("slctCritCn", "").strip(),
            "지원내용": node.findtext("alwServCn", "").strip(),
            "지원주기": node.findtext("sprtCycNm", ""),
            "지급방식": node.findtext("srvPvsnNm", ""),
        }
    except ET.ParseError:
        # XML 형식이 잘못된 경우
        return None

# --- 메인 실행 로직 ---
def process_and_merge_files():
    # 1. XML 파일 목록 찾기
    xml_files = glob.glob(XML_PATTERN)
    if not xml_files:
        print(f"❌ 오류: 폴더에서 '{XML_PATTERN}' 패턴에 맞는 XML 파일을 찾을 수 없습니다.")
        return

    print(f"📁 총 {len(xml_files)}개의 XML 파일을 찾았습니다. 파싱을 시작합니다...")

    # 2. 각 XML 파일을 파싱하여 데이터 리스트 생성
    recovered_data = []
    for file_path in tqdm(xml_files, desc="XML 파일 처리 중"):
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                content = f.read()
                # 파일 내용이 비어있지 않은 경우에만 파싱 시도
                if content.strip():
                    data = parse_service_detail(content)
                    if data:
                        recovered_data.append(data)
        except Exception as e:
            print(f"⚠️ 경고: '{file_path}' 파일 처리 중 오류 발생. 건너뜁니다. ({e})")
            
    if not recovered_data:
        print("❌ 오류: XML 파일에서 유효한 데이터를 추출하지 못했습니다.")
        return
        
    recovered_df = pd.DataFrame(recovered_data)
    print(f"✅ {len(recovered_df)}개의 서비스 데이터를 XML 파일로부터 성공적으로 복구했습니다.")

    # 3. 기존 합본 CSV 파일 불러오기
    try:
        print(f"\n기존 합본 파일 '{MAIN_CSV_FILE}'을 불러옵니다...")
        main_df = pd.read_csv(MAIN_CSV_FILE)
    except FileNotFoundError:
        print(f"❌ 오류: '{MAIN_CSV_FILE}' 파일을 찾을 수 없습니다. 파일이 스크립트와 같은 폴더에 있는지 확인하세요.")
        return

    # 4. 복구된 데이터와 기존 데이터 병합
    print("복구된 데이터와 기존 데이터를 병합합니다...")
    final_df = pd.concat([main_df, recovered_df], ignore_index=True)
    
    initial_rows = len(final_df)
    print(f"  - 병합 후 총 행 개수: {initial_rows}개")

    # 5. 서비스ID 기준으로 중복 데이터 제거 (복구된 데이터가 우선순위를 가짐)
    final_df.drop_duplicates(subset=['서비스ID'], keep='last', inplace=True)
    final_rows = len(final_df)
    print(f"  - 중복 제거 후 최종 행 개수: {final_rows}개 ({initial_rows - final_rows}개 중복 제거)")

    # 6. 최종 결과를 새 파일로 저장
    try:
        final_df.to_csv(FINAL_OUTPUT_FILE, index=False, encoding='utf-8-sig')
        print(f"\n🎉 성공! 최종 통합된 데이터를 '{FINAL_OUTPUT_FILE}' 파일로 저장했습니다.")
    except Exception as e:
        print(f"\n❌ 오류: 최종 파일을 저장하는 중 문제가 발생했습니다: {e}")

# --- 스크립트 실행 ---
if __name__ == "__main__":
    process_and_merge_files()

📁 총 89개의 XML 파일을 찾았습니다. 파싱을 시작합니다...


XML 파일 처리 중: 100%|██████████| 89/89 [00:01<00:00, 87.97it/s] 

✅ 88개의 서비스 데이터를 XML 파일로부터 성공적으로 복구했습니다.

기존 합본 파일 '중앙부처_1,2합본.csv'을 불러옵니다...
복구된 데이터와 기존 데이터를 병합합니다...
  - 병합 후 총 행 개수: 278개
  - 중복 제거 후 최종 행 개수: 273개 (5개 중복 제거)

🎉 성공! 최종 통합된 데이터를 '최종_중앙부처_통합본.csv' 파일로 저장했습니다.


중앙부처 합본-3차

In [62]:
import pandas as pd

files = ['중앙부처_1.csv', '중앙부처_2.csv']

dfs = [pd.read_csv(f) for f in files]
merged_df = pd.concat(dfs, ignore_index=True)

merged_df.to_csv("중앙부처_1,2합본.csv", index=False, encoding='utf-8-sig')

print("✅ 통합 완료 → 중앙부처_1,2합본.csv")
print(f"총 행 개수: {len(merged_df)}")
print(f"컬럼: {list(merged_df.columns)}")

✅ 통합 완료 → 중앙부처_1,2합본.csv
총 행 개수: 190
컬럼: ['서비스ID', '서비스명', '소관부처', '서비스개요', '지원대상상세', '선정기준', '지원내용', '지원주기', '지급방식']


# 지자체 파일 크롤링-복지로사이트

In [45]:
import time
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 입력 CSV 파일
MASTER_CSV = "지자체_복지서비스리스트_통합.csv"
SERVICE_ID_COLUMN = "서비스ID"

# URL 포맷
URL_TEMPLATE = "https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId={}&wlfareInfoReldBztpCd=02"

def scrape_service(driver, service_id):
    """단일 서비스 ID 크롤링"""
    url = URL_TEMPLATE.format(service_id)
    details = {
        "서비스ID": service_id,
        "카테고리": "",
        "지원대상": "",
        "선정기준": "",
        "서비스내용": "",
        "신청방법": ""
    }

    try:
        driver.get(url)

        # 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.cl-text"))
        )
        time.sleep(0.5)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # 카테고리 추출
        categories = []
        category_divs = soup.find_all("div", class_="cl-text", style=lambda x: x and "vertical-align:inherit" in x)
        for div in category_divs:
            txt = div.get_text(strip=True)
            if txt not in ["지원대상", "선정기준", "서비스 내용", "신청방법"]:  # 본문 키 제외
                categories.append(txt)
        details["카테고리"] = ", ".join(categories)

        # 본문 추출
        mapping = {
            "지원대상": "지원대상",
            "선정기준": "선정기준",
            "서비스 내용": "서비스내용",
            "신청방법": "신청방법"
        }
        for title_div in soup.find_all("div", class_="cl-text"):
            title = title_div.get_text(strip=True)
            if title in mapping:
                content_div = title_div.find_next("div", style=lambda x: x and "overflow:hidden" in x)
                if content_div:
                    details[mapping[title]] = content_div.get_text(" ", strip=True)

    except Exception as e:
        print(f"❌ 오류 (ID: {service_id}): {e}")

    return details


def get_resume_point(batch_prefix="지자체_복지서비스_크롤링결과_batch"):
    """마지막 저장된 배치 파일을 찾아 이어하기 시작점 반환"""
    batch_files = [f for f in os.listdir() if f.startswith(batch_prefix) and f.endswith(".csv")]
    if not batch_files:
        return 0, 1  # (시작 index, 다음 batch 번호)

    batch_files.sort()
    last_file = batch_files[-1]
    print(f"📂 마지막 저장된 파일: {last_file}")

    try:
        df_last = pd.read_csv(last_file)
        if not df_last.empty:
            last_id = df_last["서비스ID"].iloc[-1]
            print(f"👉 마지막 저장된 서비스ID: {last_id}")
            return df_last.index[-1] + 1 + (len(batch_files)-1)*100, len(batch_files)+1
    except Exception as e:
        print(f"⚠️ 이어하기 파일 분석 중 오류: {e}")

    return 0, 1


def main(batch_size=100):
    # 서비스ID 목록 불러오기
    df_master = pd.read_csv(MASTER_CSV)
    service_ids = df_master[SERVICE_ID_COLUMN].dropna().astype(str).tolist()

    # 이어하기 시작점 확인
    start_idx, batch_num = get_resume_point()
    if start_idx >= len(service_ids):
        print("🎉 모든 서비스ID에 대한 크롤링이 이미 완료되었습니다.")
        return

    # 크롬 드라이버 준비
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    last_id = None

    for idx, sid in enumerate(service_ids[start_idx:], start=start_idx+1):
        print(f"[{idx}/{len(service_ids)}] 크롤링 중: {sid}")
        data = scrape_service(driver, sid)
        results.append(data)
        last_id = sid

        # 100개 단위 저장
        if idx % batch_size == 0:
            out_df = pd.DataFrame(results)
            out_df.to_csv(f"지자체_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
            print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")
            batch_num += 1
            results = []

        time.sleep(0.5)  # 서버 부하 방지

    # 마지막 남은 결과 저장
    if results:
        out_df = pd.DataFrame(results)
        out_df.to_csv(f"지자체_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")

    driver.quit()
    print(f"✅ 마지막 저장된 서비스ID: {last_id}")


if __name__ == "__main__":
    main(batch_size=100)


[1/4555] 크롤링 중: WLF00006172
[2/4555] 크롤링 중: WLF00005696
[3/4555] 크롤링 중: WLF00006171
[4/4555] 크롤링 중: WLF00006170
[5/4555] 크롤링 중: WLF00004946
[6/4555] 크롤링 중: WLF00006112
[7/4555] 크롤링 중: WLF00002178
[8/4555] 크롤링 중: WLF00005687
[9/4555] 크롤링 중: WLF00005691
[10/4555] 크롤링 중: WLF00006169
[11/4555] 크롤링 중: WLF00000300
[12/4555] 크롤링 중: WLF00004006
[13/4555] 크롤링 중: WLF00004192
[14/4555] 크롤링 중: WLF00000759
[15/4555] 크롤링 중: WLF00002772
[16/4555] 크롤링 중: WLF00002751
[17/4555] 크롤링 중: WLF00004099
[18/4555] 크롤링 중: WLF00006168
[19/4555] 크롤링 중: WLF00002747
[20/4555] 크롤링 중: WLF00001818
[21/4555] 크롤링 중: WLF00000578
[22/4555] 크롤링 중: WLF00003632
[23/4555] 크롤링 중: WLF00003799
[24/4555] 크롤링 중: WLF00004406
[25/4555] 크롤링 중: WLF00004131
[26/4555] 크롤링 중: WLF00001009
[27/4555] 크롤링 중: WLF00006165
[28/4555] 크롤링 중: WLF00005908
[29/4555] 크롤링 중: WLF00005769
[30/4555] 크롤링 중: WLF00005683
[31/4555] 크롤링 중: WLF00005460
[32/4555] 크롤링 중: WLF00005092
[33/4555] 크롤링 중: WLF00004387
[34/4555] 크롤링 중: WLF00000904
[35/4555] 크롤링 중: WLF000

지자체 크롤링 통합

In [46]:
import pandas as pd
import glob

# 합칠 파일 패턴 지정
file_list = sorted(glob.glob("지자체_복지서비스_크롤링결과_batch*.csv"))

print(f"총 {len(file_list)}개 파일을 병합합니다.")

# 파일 읽어서 합치기
dfs = [pd.read_csv(f) for f in file_list]
merged_df = pd.concat(dfs, ignore_index=True)

# 최종 저장
merged_df.to_csv("지자체_복지서비스_크롤링결과_통합본.csv", index=False, encoding="utf-8-sig")

print("✅ 병합 완료: 지자체_복지서비스_크롤링결과_통합본.csv")



총 46개 파일을 병합합니다.
✅ 병합 완료: 지자체_복지서비스_크롤링결과_통합본.csv


지자체 csv 통합 - 세부내용 + 리스트

In [47]:
import pandas as pd

# 파일 불러오기
base_df = pd.read_csv("지자체_복지서비스리스트_통합.csv", encoding="utf-8-sig")
crawl_df = pd.read_csv("지자체_복지서비스_크롤링결과_통합본.csv", encoding="utf-8-sig")

# 서비스ID 기준 병합 (좌측 기준)
merged_df = pd.merge(base_df, crawl_df, on="서비스ID", how="left")

# 저장 (최종 파일 이름 지정)
output_file = "local_final_welfare_list.csv"
merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 병합 완료: {output_file}")
print(f"총 행 수: {len(merged_df)}")


✅ 병합 완료: local_final_welfare_list.csv
총 행 수: 4555


민간복지 - 크롤링

In [57]:
import requests
import pandas as pd
import time

URL = "https://www.bokjiro.go.kr/ssis-tbu/TWAT52005M/twataa/wlfareInfo/selectWlfareInfo.do"

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    "Referer": "https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52005M.do",
    "Content-Type": "application/json;charset=UTF-8",
}

def fetch_page(page: int):
    payload = {
        "dmSearchParam": {
            "page": str(page),
            "orderBy": "date",
            "tabId": "3",   # ✅ 민간 탭
            "searchTerm": "",
            "onlineYn": "",
            "bkjrLftmCycCd": "",
            "daesang": "",
            "period": "",
            "age": "",
            "region": "",
            "jjim": "",
            "subject": "",
            "favoriteKeyword": "Y",
            "sido": "",
            "gungu": "",
            "endYn": "N"
        },
        "dmScr": {
            "curScrId": "tbu/app/twat/twata/twataa/TWAT52005M",
            "befScrId": ""
        }
    }
    res = requests.post(URL, headers=headers, json=payload)
    res.raise_for_status()
    return res.json().get("dsServiceList3", [])

# ✅ 전체 페이지 크롤링
all_data = []
for page in range(1, 39):  # 1~38 페이지
    data = fetch_page(page)
    print(f"{page}페이지: {len(data)}건 수집")
    all_data.extend(data)
    time.sleep(0.3)

print("총 민간 데이터:", len(all_data))

# ✅ 기본 필드 + RETURN_STR 세부항목 파싱
rows = []
for svc in all_data:
    base_info = {
        "서비스ID": svc.get("WLFARE_INFO_ID"),
        "서비스명": svc.get("WLFARE_INFO_NM"),
        "서비스 요약": svc.get("WLFARE_INFO_OUTL_CN"),
        "상세 링크": f"https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52011M.do?wlfareInfoId={svc.get('WLFARE_INFO_ID')}&wlfareInfoReldBztpCd=03",
        "소관기관": svc.get("BIZ_CHR_INST_NM"),
        "대표연락처": svc.get("RPRS_CTADR"),
        "신청가능여부": svc.get("ONLINEYN"),
        "주소": svc.get("ADDR"),
        "태그": svc.get("TAG_NM"),
        "시작일": svc.get("ENFC_BGNG_YMD"),
        "종료일": svc.get("ENFC_END_YMD"),
        "진행상태": svc.get("CVL_PROGRSS_STATUS"),
    }
    
    # RETURN_STR 파싱
    return_str = svc.get("RETURN_STR", "")
    parsed_info = {}
    if return_str:
        for item in return_str.split(";"):
            if ":" in item:
                k, v = item.split(":", 1)
                parsed_info[k.strip()] = v.strip()
    
    # 기본정보 + RETURN_STR 확장정보 병합
    rows.append({**base_info, **parsed_info})

# ✅ CSV 저장
df = pd.DataFrame(rows)
df.to_csv("민간_복지서비스_확장.csv", index=False, encoding="utf-8-sig")

print("📁 민간_복지서비스_확장.csv 저장 완료")


1페이지: 9건 수집
2페이지: 9건 수집
3페이지: 9건 수집
4페이지: 9건 수집
5페이지: 9건 수집
6페이지: 9건 수집
7페이지: 9건 수집
8페이지: 9건 수집
9페이지: 9건 수집
10페이지: 9건 수집
11페이지: 9건 수집
12페이지: 9건 수집
13페이지: 9건 수집
14페이지: 9건 수집
15페이지: 9건 수집
16페이지: 9건 수집
17페이지: 9건 수집
18페이지: 9건 수집
19페이지: 9건 수집
20페이지: 9건 수집
21페이지: 9건 수집
22페이지: 9건 수집
23페이지: 9건 수집
24페이지: 9건 수집
25페이지: 9건 수집
26페이지: 9건 수집
27페이지: 9건 수집
28페이지: 9건 수집
29페이지: 9건 수집
30페이지: 9건 수집
31페이지: 9건 수집
32페이지: 9건 수집
33페이지: 9건 수집
34페이지: 9건 수집
35페이지: 9건 수집
36페이지: 9건 수집
37페이지: 9건 수집
38페이지: 3건 수집
총 민간 데이터: 336
📁 민간_복지서비스_확장.csv 저장 완료


# 민간서비스 상세내역 크롤링

In [64]:
import time
import os
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 입력 CSV 파일
MASTER_CSV = "민간_복지서비스_확장.csv"   # 민간 서비스 ID 들어있는 파일
SERVICE_ID_COLUMN = "서비스ID"

# URL 포맷
URL_TEMPLATE = "https://www.bokjiro.go.kr/ssis-tbu/twataa/wlfareInfo/moveTWAT52015M.do?wlfareInfoId={}&wlfareInfoReldBztpCd=03"

def scrape_service(driver, service_id):
    """단일 민간 서비스 ID 상세 크롤링"""
    url = URL_TEMPLATE.format(service_id)
    details = {
        "서비스ID": service_id,
        "카테고리": "",
        "사업상태": "",
        "사업기간": "",
        "연락처": "",
        "이메일": "",
        "사업목적": "",
        "지원대상": "",
        "지원내용": "",
        "신청방법": "",
        "제출서류": ""
    }

    try:
        driver.get(url)

        # 로딩 대기
        WebDriverWait(driver, 10).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, "div.cl-text"))
        )
        time.sleep(0.5)

        soup = BeautifulSoup(driver.page_source, "html.parser")

        # ✅ 카테고리 (상단 badge-wlfare 영역)
        category_divs = soup.select("div.badge-wlfare div.cl-text")
        categories = [c.get_text(strip=True) for c in category_divs if c.get_text(strip=True)]
        details["카테고리"] = ", ".join(categories)

        # ✅ 단일 필드 (사업상태, 사업기간, 연락처, 이메일)
        mapping_simple = {
            "사업상태": "사업상태",
            "사업기간": "사업기간",
            "연락처": "연락처",
            "이메일": "이메일"
        }
        for title, col in mapping_simple.items():
            title_div = soup.find("div", class_="cl-text", string=title)
            if title_div:
                sibling_div = title_div.find_parent().find_next_sibling("div")
                if sibling_div and "cl-text" in sibling_div.get("class", []):
                    details[col] = sibling_div.get_text(" ", strip=True)

        # ✅ 본문 필드 (사업목적, 지원대상, 지원내용, 신청방법, 제출서류)
        mapping_long = {
            "사업목적": "사업목적",
            "지원대상": "지원대상",
            "지원내용": "지원내용",
            "신청방법": "신청방법",
            "제출서류": "제출서류"
        }
        for title, col in mapping_long.items():
            title_div = soup.find("div", class_="cl-text", string=title)
            if title_div:
                content_block = title_div.find_parent().find_next("div", class_="cl-control")
                if content_block:
                    text_div = content_block.find("div", class_="cl-text")
                    if text_div:
                        details[col] = text_div.get_text("\n", strip=True)

    except Exception as e:
        print(f"❌ 오류 (ID: {service_id}): {e}")

    return details


def get_resume_point(batch_prefix="민간_복지서비스_크롤링결과_batch"):
    """마지막 저장된 배치 파일을 찾아 이어하기 시작점 반환"""
    batch_files = [f for f in os.listdir() if f.startswith(batch_prefix) and f.endswith(".csv")]
    if not batch_files:
        return 0, 1  # (시작 index, 다음 batch 번호)

    batch_files.sort()
    last_file = batch_files[-1]
    print(f"📂 마지막 저장된 파일: {last_file}")

    try:
        df_last = pd.read_csv(last_file)
        if not df_last.empty:
            last_id = df_last["서비스ID"].iloc[-1]
            print(f"👉 마지막 저장된 서비스ID: {last_id}")
            return df_last.index[-1] + 1 + (len(batch_files)-1)*100, len(batch_files)+1
    except Exception as e:
        print(f"⚠️ 이어하기 파일 분석 중 오류: {e}")

    return 0, 1


def main(batch_size=100):
    # 서비스ID 목록 불러오기
    df_master = pd.read_csv(MASTER_CSV)
    service_ids = df_master[SERVICE_ID_COLUMN].dropna().astype(str).tolist()

    # 이어하기 시작점 확인
    start_idx, batch_num = get_resume_point()
    if start_idx >= len(service_ids):
        print("🎉 모든 서비스ID에 대한 크롤링이 이미 완료되었습니다.")
        return

    # 크롬 드라이버 준비
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    service = ChromeService(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)

    results = []
    last_id = None

    for idx, sid in enumerate(service_ids[start_idx:], start=start_idx+1):
        print(f"[{idx}/{len(service_ids)}] 크롤링 중: {sid}")
        data = scrape_service(driver, sid)
        results.append(data)
        last_id = sid

        # 100개 단위 저장
        if idx % batch_size == 0:
            out_df = pd.DataFrame(results)
            out_df.to_csv(f"민간_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
            print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")
            batch_num += 1
            results = []

        time.sleep(0.5)  # 서버 부하 방지

    # 마지막 남은 결과 저장
    if results:
        out_df = pd.DataFrame(results)
        out_df.to_csv(f"민간_복지서비스_크롤링결과_batch{batch_num}.csv", index=False, encoding="utf-8-sig")
        print(f"💾 Batch {batch_num} 저장 완료 ({len(results)}개)")

    driver.quit()
    print(f"✅ 마지막 저장된 서비스ID: {last_id}")


if __name__ == "__main__":
    main(batch_size=100)


[1/336] 크롤링 중: BOK00001001
[2/336] 크롤링 중: BOK00000995
[3/336] 크롤링 중: BOK00000994
[4/336] 크롤링 중: BOK00000992
[5/336] 크롤링 중: BOK00000945
[6/336] 크롤링 중: BOK00000982
[7/336] 크롤링 중: BOK00000987
[8/336] 크롤링 중: BOK00000981
[9/336] 크롤링 중: BOK00000979
[10/336] 크롤링 중: BOK00000978
[11/336] 크롤링 중: BOK00000977
[12/336] 크롤링 중: BOK00000947
[13/336] 크롤링 중: BOK00000974
[14/336] 크롤링 중: BOK00000971
[15/336] 크롤링 중: BOK00000953
[16/336] 크롤링 중: BOK00000956
[17/336] 크롤링 중: BOK00000958
[18/336] 크롤링 중: BOK00000948
[19/336] 크롤링 중: BOK00000820
[20/336] 크롤링 중: BOK00000959
[21/336] 크롤링 중: BOK00000941
[22/336] 크롤링 중: BOK00000883
[23/336] 크롤링 중: BOK00000939
[24/336] 크롤링 중: BOK00000926
[25/336] 크롤링 중: BOK00000925
[26/336] 크롤링 중: BOK00000922
[27/336] 크롤링 중: BOK00000920
[28/336] 크롤링 중: BOK00000918
[29/336] 크롤링 중: S0000248225
[30/336] 크롤링 중: S9765800008
[31/336] 크롤링 중: S0038597837
[32/336] 크롤링 중: S9765742345
[33/336] 크롤링 중: S0041108420
[34/336] 크롤링 중: S9765686049
[35/336] 크롤링 중: S9765853441
[36/336] 크롤링 중: S9765870934
[

파일 합치기

In [65]:
import pandas as pd
import glob

# 병합할 파일 패턴
file_pattern = "민간_복지서비스_크롤링결과_batch*.csv"

# 모든 배치 파일 경로 가져오기
files = sorted(glob.glob(file_pattern))

print("📂 합칠 파일:", files)

# 파일 읽어서 세로로 이어붙이기
dfs = [pd.read_csv(f) for f in files]
merged_df = pd.concat(dfs, ignore_index=True)

# 저장
output_file = "민간_복지서비스_크롤링결과_total.csv"
merged_df.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 저장 완료: {output_file}, 총 {len(merged_df)} 행")


📂 합칠 파일: ['민간_복지서비스_크롤링결과_batch1.csv', '민간_복지서비스_크롤링결과_batch2.csv', '민간_복지서비스_크롤링결과_batch3.csv', '민간_복지서비스_크롤링결과_batch4.csv']
✅ 저장 완료: 민간_복지서비스_크롤링결과_total.csv, 총 336 행


In [67]:
import pandas as pd

# 파일 경로
base_file = "민간_복지서비스_확장.csv"          # 기준 파일
crawl_file = "민간_복지서비스_크롤링결과_total.csv"  # 크롤링 결과 파일
output_file = "private_final_welfare_list.csv"

# CSV 읽기
df_base = pd.read_csv(base_file)
df_crawl = pd.read_csv(crawl_file)

# 기준 컬럼명 확인
print("기준 파일 컬럼:", df_base.columns.tolist())
print("크롤링 파일 컬럼:", df_crawl.columns.tolist())

# 서비스ID 기준으로 병합 (오른쪽 방향으로 붙이기)
df_merged = pd.merge(df_base, df_crawl, on="서비스ID", how="left")

# 저장
df_merged.to_csv(output_file, index=False, encoding="utf-8-sig")

print(f"✅ 병합 완료! 저장된 파일: {output_file}")


기준 파일 컬럼: ['서비스ID', '서비스명', '서비스 요약', '상세 링크', '소관기관', '대표연락처', '신청가능여부', '주소', '태그', '시작일', '종료일', '진행상태', 'INTRS_THEMA_CD', 'FMLY_CIRC_CD', 'BKJR_LFTM_CYC_CD', 'WLFARE_INFO_AGGRP_CD']
크롤링 파일 컬럼: ['서비스ID', '카테고리', '사업상태', '사업기간', '연락처', '이메일', '사업목적', '지원대상', '지원내용', '신청방법', '제출서류']
✅ 병합 완료! 저장된 파일: private_final_welfare_list.csv
